In [ ]:
import pandas as pd
import json
import csv
from neo4j import GraphDatabase

# Query from reactome using neo4j

In [ ]:

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.__uri = uri
        self.__user = user
        self.__password = password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        with self.__driver.session(database=db) as session:
            result = session.run(query, parameters)
            return [record for record in result]


In [ ]:

def flatten_node_data(records):
    """Extracts and flattens data from Neo4j records into a list of dictionaries."""
    data = []
    for record in records:
        node = record['n']
        flat_data = dict(node)
        flat_data['element_id'] = node.id
        data.append(flat_data)
    return data

def save_to_json(records, json_file_path):
    """Converts flattened Neo4j query results to a JSON file."""
    flattened_data = flatten_node_data(records)
    with open(json_file_path, 'w') as f:
        json.dump(flattened_data, f, indent=4)


def save_to_csv(records, csv_file_path=None):
    """
    Converts Neo4j query results to a pandas DataFrame and optionally saves to a CSV file.
    
    :param records: List of Neo4j Record objects.
    :param csv_file_path: Optional path to save the CSV file. If None, no CSV will be saved.
    :return: pandas DataFrame containing the extracted data.
    """
    data = flatten_node_data(records)
    df = pd.DataFrame(data)
    
    if csv_file_path:
        df.to_csv(csv_file_path, index=False)
    
    return df

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out
    
def json_to_csv(json_file_path, csv_file_path):
    # Load JSON data
    with open(json_file_path) as json_file:
        data = json.load(json_file)

    # Flatten the JSON data
    flattened_data = [flatten_json(item) for item in data]

    # Open a CSV file for writing
    with open(csv_file_path, mode='w', newline='') as csv_file:
        # Create a CSV writer object
        writer = csv.writer(csv_file)

        # Write the header if flattened_data is not empty
        if flattened_data:
            header = flattened_data[0].keys()
            writer.writerow(header)

            # Write the data rows
            for item in flattened_data:
                writer.writerow(item.values())


In [ ]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "neo4j"
query = "MATCH (n) WHERE n.speciesName = 'Mus musculus' RETURN n"


# Establish connection
conn = Neo4jConnection(uri, user, password)
results = conn.query(query)

save_to_csv(results,'mice.csv')

# Close the connection
conn.close()

In [ ]:
mice_df = pd.read_csv('mice.csv')

In [ ]:
mice_df